In [ ]:
!pip install transformers
!pip install huggingfacehub
!pip install torch

ERROR: Could not find a version that satisfies the requirement huggingfacehub (from versions: none)
ERROR: No matching distribution found for huggingfacehub
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x

In [ ]:
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 5.3 MB/s eta 0:00:00


# **Sentiment Analysis**

### Import

In [ ]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import os
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import numpy as np


In [ ]:
#import test data and target
test_text = open('test_text.txt').read().split("\n")[:-1]
train_labels_str = open('test_labels.txt').read().split("\n")[:-1]
test_labels = [int(x) for x in train_labels_str]

In [ ]:
os.environ['HF_TOKEN'] = 'xxx'
os.environ['HUGGINGFACEHUB_API_TOKEN'] = 'xxx'

### Setup

In [ ]:
# Check if a GPU is available
if torch.cuda.is_available():
    print("GPU is available")
else:
    print("GPU is not available")

### Build the Llama-Classifier

In [ ]:
# Load the Meta-Llama-3-8B model
model_name = "meta-llama/Meta-Llama-3-8B"
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Move the model to the GPU
model.to("cuda")

In [ ]:
# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
# Define the sentiment labels
sentiment_labels = ["negative", "positive", "neutral"]

# Tokenize the input text (using the GPU if available)
inputs = tokenizer(test_text, padding=True, truncation=True, return_tensors="pt").to("cuda")

# Enable mixed precision training
scaler = torch.cuda.amp.GradScaler()

# Predict sentiment (using the GPU)
with torch.cuda.amp.autocast():
    outputs = model(**inputs)
predictions = outputs.logits.argmax(-1)

# Map predicted class indices to sentiment labels
predicted_labels = [sentiment_labels[prediction.item()] for prediction in predictions]

# Print the predicted sentiment labels
for text, predicted_label in zip(test_text, predicted_labels):
    print(f"Text: {text}")
    print(f"Sentiment: {predicted_label}")
    print()

### Evalutation

In [ ]:
def translate_predictions(predictions):
    translation_dict = {"negative": 0, "neutral": 1, "positive": 2}
    translated_list = [translation_dict[prediction] for prediction in predictions]
    return translated_list

# Example usage:
translated_predictions = translate_predictions(predicted_label)

In [ ]:
def generate_confusion_matrix(test_labe, predicted_labels, labels):
    cm = confusion_matrix(actual_labels, predicted_labels, labels=labels)
    return cm

cm = generate_confusion_matrix(test_labels, translated_predictions, [0,1,2])

In [ ]:
def print_classification_metrics(actual_labels, predicted_labels, labels):
    report = classification_report(actual_labels, predicted_labels, labels=labels)
    print("Classification Report:")
    print(report)

print_classification_metrics(test_labels, translated_predictions, [0,1,2])